# Calculate GWP and make tabels for the paper...

Description of the experiments. bla bla

To do:
- Add references  https://jupyterbook.org/en/stable/content/citations.html
- Write a detailed explanation of the method inkluding references

In [1]:
import numpy as np
import pandas as pd

Input and output path:

In [2]:
path = r"/mnt/c/Users/ragnhibs/OneDrive - CICERO senter for klimaforskning/Documents/Hydrogen/Hydrogen_prosjektet/GWP_calc/GWP_notebook/input/"
outputpath= r"/mnt/c/Users/ragnhibs/OneDrive - CICERO senter for klimaforskning/Documents/Hydrogen/Hydrogen_prosjektet/GWP_calc/GWP_notebook/output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: AR5	
agwp100_CO2 = 0.09170

#CH4 tau_strat[yr] Source: ?
tau_strat = 120.0

#CH4 tau_soil [yr] Source: ?
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44300

Add possibility to adjust the dry-deposition
QUESTION: Adjust the perturbations with the same relative factors as in the control?

In [4]:
adjust_drydep = True
if(adjust_drydep):
    drydep = 80.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'
    #print(outputpath)


## Read input data

To run this notebook, preprosessed output from the 3 set of simulations and put into a folder.

## Hydrogen results

### H2 burden [Tg]:

In [5]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden = df_h2_burden.append(delta)
df_h2_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,196.210864,194.3987,194.4,191.7
10H2,214.861270,212.9990,213.0,655.6
10CH4,196.785571,194.9480,194.9,NaN
deltaH2,18.650406,18.6003,18.6,463.9


### H2 loss

### H2 drydep [Tg/yr]

In [6]:
file = 'H2_drydep.txt'
df_h2_drydep = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,59.518555,72.45730,52.06,54.66
10H2,65.470284,79.69643,57.23,203.65
10CH4,59.518572,72.47052,52.07,NaN


In [7]:
if(adjust_drydep):
    print('NB drydep adjusted')
    
    #Adjust by the relative adjustment in the control simulations
    adjust = drydep/df_h2_drydep.loc['CTRL']
    df_h2_drydep = df_h2_drydep*adjust
    
    print(df_h2_drydep)

NB drydep adjusted
           OSLOCTM3  WACCM6-2deg       INCA    GFDL-emi
Scenario                                               
CTRL      80.000000    80.000000  80.000000   80.000000
10H2      87.999831    87.992713  87.944679  298.060739
10CH4     80.000023    80.014596  80.015367         NaN


### H2 atm. loss [Tg/yr]

In [8]:
file = 'H2_atm_loss.txt'
df_h2_atmloss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,28.389650,29.51206,22.1256,22.1241
10H2,30.973757,32.19348,24.1501,69.4454
10CH4,27.635640,28.60070,21.6251,NaN


### H2 total loss [Tg/yr]:

In [9]:
df_h2_loss = df_h2_atmloss + df_h2_drydep
df_h2_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,108.389650,109.512060,102.125600,102.124100
10H2,118.973588,120.186193,112.094779,367.506139
10CH4,107.635663,108.615296,101.640467,NaN


### H2 prod

### H2 atm. prod [Tg/yr]

In [10]:
file = 'H2_atm_prod.txt'
df_h2_atmprod = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,56.323356,34.62987,46.6780,44.6840
10H2,56.234898,34.52268,46.6137,43.1153
10CH4,59.160453,37.10409,48.9050,NaN


### Estimated emissions (Total loss = Total prod + emission)

In [11]:
df_h2_estemis = df_h2_atmloss.loc['CTRL'] + df_h2_drydep.loc['CTRL'] - df_h2_atmprod.loc['CTRL']
df_h2_estemis


OSLOCTM3       52.066294
WACCM6-2deg    74.882190
INCA           55.447600
GFDL-emi       57.440100
Name: CTRL, dtype: float64

### H2 surface concentration [ppb]

In [12]:
file = 'H2_surfconc.txt'
df_h2_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc.loc['10H2']-df_h2_surfconc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc = df_h2_surfconc.append(delta)
df_h2_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,531.7,531.775,531.8,533.6
10H2,584.9,584.952,585.0,1880.1
deltaH2,53.2,53.177,53.2,1346.5


### H2 lifetime [yr]

In [13]:
df_h2_lifetime = df_h2_burden.drop('deltaH2')/df_h2_loss
df_h2_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,1.810236,1.775135,1.903538,1.877128
10H2,1.805958,1.772242,1.900178,1.783916
10CH4,1.828256,1.794848,1.917543,NaN


### H2 atmospheric lifetime [yr]

In [14]:
df_h2_atm_lifetime = df_h2_burden.drop('deltaH2')/df_h2_atmloss
df_h2_atm_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,6.911352,6.587094,8.786202,8.664759
10H2,6.936881,6.616215,8.819839,9.440510
10CH4,7.120717,6.816197,9.012675,NaN


### H2 flux (burden dived by lifetime) [Tg/yr]

Add a description of the delta flux here.

In [15]:
df_h2_flux = df_h2_burden.drop('deltaH2')/df_h2_lifetime
#Add delta flux 10H2:
delta = df_h2_flux.loc['10H2']-df_h2_flux.loc['CTRL']
delta.name = 'deltaH2'
df_h2_flux = df_h2_flux.append(delta)
#Add delta flux 10CH4:
delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_h2_flux = df_h2_flux.append(delta)
df_h2_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,108.389650,109.512060,102.125600,102.124100
10H2,118.973588,120.186193,112.094779,367.506139
10CH4,107.635663,108.615296,101.640467,NaN
deltaH2,10.583938,10.674133,9.969179,265.382039
deltaCH4,-0.753987,-0.896764,-0.485133,NaN


## Methane results

### CH4 burden [Tg]

In [16]:
file = 'CH4_burden.txt'
df_ch4_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden.loc['10CH4']-df_ch4_burden.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden = df_ch4_burden.append(delta)
df_ch4_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,4994.964696,5006.379,4994.5,4935.0
10H2,4995.030494,5006.521,4994.6,4935.0
10CH4,5495.073931,5508.193,5494.0,NaN
deltaCH4,500.109236,501.814,499.5,NaN


### CH4 atmospheric loss [Tg/yr]

In [17]:
file = 'CH4_loss.txt'
df_ch4_loss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,682.972311,742.3324,549.5713,582.2586
10H2,680.300007,738.9528,547.4653,537.0809
10CH4,727.967996,788.0606,587.6182,NaN


### CH4 surface concentration [ppb]

In [18]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc.loc['10CH4']-df_ch4_surfconc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc = df_ch4_surfconc.append(delta)
df_ch4_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,1813.1,1807.800,1813.2,1803.6
10CH4,1994.4,1988.634,1994.5,NaN
deltaCH4,181.3,180.834,181.3,NaN


### CH4 lifetime [yr]

In [19]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime



,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,7.313568,6.744120,9.087993,8.475615
10H2,7.342394,6.775157,9.123135,9.188560
10CH4,7.548510,6.989555,9.349608,NaN


### Total CH4 lifetime [yr]

In [20]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,6.608709,6.140219,8.024482,7.543249
10H2,6.632237,6.165935,8.051869,8.102788
10CH4,6.799955,6.343006,8.227765,NaN


### CH4 flux (burden dived by total lifetime) [Tg/yr]

In [21]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
#Add delta CH4 flux 10H2
delta = df_ch4_flux.loc['10H2']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaH2'
df_ch4_flux = df_ch4_flux.append(delta)
#Add delta CH4 flux 10CH4
delta = df_ch4_flux.loc['10CH4']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_flux = df_ch4_flux.append(delta)
df_ch4_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
CTRL,755.815546,815.342094,622.407758,654.22735
10H2,753.144202,811.964565,620.303217,609.04965
10CH4,808.104491,868.388415,667.739033,NaN
deltaH2,-2.671344,-3.377529,-2.104542,-45.17770
deltaCH4,52.288944,53.046321,45.331275,NaN


## Ozone results

### Ozone delta DU troposphere

In [22]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
10H2,0.038463,0.027579,0.029285,0.729164
10CH4,0.830869,0.859612,0.539447,0.762391


### Ozone delta DU strat

In [23]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
10H2,0.009863,-0.012748,-0.027429,-0.226384
10CH4,0.829367,0.773137,0.352413,1.072883


### Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone concentration.For GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4 pert minus the H2 pert.

In [24]:
file = 'ozone_rf.txt'
df_ozone_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
Scenario,,,,
10H2,1.8707,1.4121,1.1167,34.0088
10CH4,40.9708,41.6999,22.5765,37.5865


### Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline. Full calculations will be performed. So far only LW. Multiply by 0.85

In [25]:
file = 'H2O_rf.txt'
df_h2o_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)

#'NBNBNB multiply these numbers by 0.85
df_h2o_rf = df_h2o_rf.multiply(0.85)
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA
Scenario,,,
10H2,1.3515,0.5185,0.3995
10CH4,9.1800,2.8050,3.4000


In [26]:
#NBNBBN not multiply these numbers by 0.85'
file = 'H2O_rf_gfdl.txt'
h2o_rf_gfdl = pd.read_csv(path + file, sep=',',index_col=0,header=0)
h2o_rf_gfdl

,GFDL
H2 [mW m-2],28.2625
H2+CH4 [mW m-2],53.1250



# Start calculations:

### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [27]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

OSLOCTM3       5.026484
WACCM6-2deg    4.981857
INCA           5.336447
GFDL-emi       5.073817
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 due to.... Why this includes feedback...

In [28]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux


OSLOCTM3       0.252396
WACCM6-2deg    0.316422
INCA           0.211105
GFDL-emi       0.170236
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [29]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.467272
WACCM6-2deg    3.408983
INCA           3.999446
GFDL-emi            NaN
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surf conc per emiss H2 [ppb yr /Tg H2]

In [30]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'

#Add GFDL:
#Increase in surface concentration CH4:
#1808 to 2005 ppbv
df_surf_ch4_per_h2_flux['GFDL-emi']= (2005.-1808.)/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_surf_ch4_per_h2_flux


OSLOCTM3       0.875126
WACCM6-2deg    1.078677
INCA           0.844302
GFDL-emi       0.742326
Name: surf_ch4_per_h2_flux, dtype: float64

### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

Why we multiply by -1

In [31]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux


OSLOCTM3       0.014420
WACCM6-2deg    0.016905
INCA           0.010702
GFDL-emi            NaN
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

Explain why this is AGWP

In [32]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'


#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'



### Initialize H2 GWP table

In [33]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
O3,NaN,NaN,NaN,NaN
CH4,NaN,NaN,NaN,NaN
strat H2O,NaN,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN,NaN


### Add methane GWP

In [34]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2

### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

Explain that RF/flux equal AGWP..

In [35]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_h2o['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2+CH4 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_h2_agwp_h2o.name = 'h2_agwp_h2o'

#NBNB GFDL methane induced is included.

In [36]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'
df_h2o_rf_per_h2_flux['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']

#Strat H2O RF per methane flux (Move this to the methane part?)
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


### Add stratospheric H2O GWP

In [37]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2


### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [38]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

#For GFDL-emi include the methane induced part.
df_h2_agwp_o3['GFDL-emi'] = (df_ozone_rf['GFDL-emi'].loc['10H2']+df_ozone_rf['GFDL-emi'].loc['10CH4'])/df_h2_flux['GFDL-emi'].loc['deltaH2']


In [39]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

In [40]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'


### Add Ozone GWP

In [41]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2


### For the per flux table

In [42]:
df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux.loc['deltaH2']
df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'


## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden, tropospheric concentration?

In [43]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'

### Add methane induced O3 GWP

In [44]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2

### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [45]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'




### Add methane induced strat H2O GWP

In [46]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2


# Methane GWP

Initialize CH4 GWP

In [47]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
O3,NaN,NaN,NaN,NaN
CH4,NaN,NaN,NaN,NaN
strat H2O,NaN,NaN,NaN,NaN
H2,NaN,NaN,NaN,NaN


### Methane AGWP100 O3 [mW m-2 yr Tg-1]

Integrated O3 from CH4 RF [W m-2 yr Tg-1]

In [48]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'

### Add ozone GWP

In [49]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 
#df_ch4_gwp

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [50]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'

### Add methane GWP

In [51]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 
#df_ch4_gwp

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [52]:
print(df_h2o_rf.loc['10CH4'])
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'



OSLOCTM3       9.180
WACCM6-2deg    2.805
INCA           3.400
Name: 10CH4, dtype: float64


### Add Strat H2O GWP:

In [53]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2
#df_ch4_gwp

### Add GWP via H2

In [54]:
df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()

# Main results and tabels

## H2 GWP 100

In [55]:
#Save to file:
#print(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.loc['total']=df_h2_gwp.sum()
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
O3,1.927469,1.442658,1.221540,2.942007
CH4,4.227707,5.211056,4.078800,3.586155
strat H2O,1.392514,0.529721,0.437007,2.183022
O3 CH4ind,2.156641,2.712545,1.146535,NaN
strat H2O CH4ind,0.483221,0.182463,0.172667,NaN
total,10.187552,10.078442,7.056549,8.711184


## CH4 GWP 100

In [56]:
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi
O3,8.544669,8.572558,5.431119,NaN
CH4,16.750290,16.468696,19.321207,NaN
strat H2O,1.914536,0.576645,0.817922,NaN
H2,0.146901,0.170379,0.075519,NaN
total,27.356395,25.788277,25.645767,0.0


## Table per flux H2

In [57]:
df_per_flux_h2 = pd.concat([df_h2_flux.loc['deltaH2'],df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,df_ch4_flux_per_h2_flux,df_ch4_rf_per_h2_flux,
                            df_trop_du_ozone_per_h2_flux,df_strat_du_ozone_per_h2_flux,
                            df_ozone_rf_per_h2_flux,df_h2o_rf_per_h2_flux],axis=1)

#Save to file:
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]'}
#Possibility to rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2

/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg]
GFDL-emi,265.382039,5.073817,0.742326,0.170236,0.328850,0.002748,-0.000853,0.128150,0.106497
INCA,9.969179,5.336447,0.844302,0.211105,0.374026,0.002938,-0.002751,0.112015,0.040074
OSLOCTM3,10.583938,5.026484,0.875126,0.252396,0.387681,0.003634,0.000932,0.176749,0.127693
WACCM6-2deg,10.674133,4.981857,1.078677,0.316422,0.477854,0.002584,-0.001194,0.132292,0.048575


## Table per flux CH4

To do: fix the table headings in the table (as for H2 above)

In [58]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],df_surf_ch4_per_ch4_flux,
                             df_h2_flux_per_ch4_flux,df_trop_du_ozone_per_ch4_flux,df_strat_du_ozone_per_ch4_flux,
                             df_ozone_rf_per_ch4_flux,df_h2o_rf_per_ch4_flux],axis=1)

#Save to file:
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')
df_per_flux_ch4

/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,trop_du_ozone_per_ch4_flux,strat_du_ozone_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux
GFDL-emi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
INCA,45.331275,3.999446,0.010702,0.011900,0.007774,0.498034,0.075003
OSLOCTM3,52.288944,3.467272,0.014420,0.015890,0.015861,0.783546,0.175563
WACCM6-2deg,53.046321,3.408983,0.016905,0.016205,0.014575,0.786104,0.052878


## H2 budget tabel

In [59]:
df_budget_h2 = pd.concat([df_h2_burden.loc['CTRL'],df_h2_lifetime.loc['CTRL'],
                          df_h2_atmloss.loc['CTRL'],df_h2_drydep.loc['CTRL'],
                          df_h2_atmprod.loc['CTRL'],df_h2_atm_lifetime.loc['CTRL'],
                          df_h2_estemis],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 lifetime [yrs]',
                        'H2 atm.loss[Tg/yr]',
                        'H2 drydep [Tg/yr]',
                        'H2 atm. prod [Tg/yr]',
                        'H2 atmospheric lifetime [yrs]',
                        'H2 estimated emissions [Tg/yr]']

df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2

,H2 burden [Tg],H2 lifetime [yrs],H2 atm.loss[Tg/yr],H2 drydep [Tg/yr],H2 atm. prod [Tg/yr],H2 atmospheric lifetime [yrs],H2 estimated emissions [Tg/yr]
OSLOCTM3,196.210864,1.810236,28.38965,80.0,56.323356,6.911352,52.066294
WACCM6-2deg,194.398700,1.775135,29.51206,80.0,34.629870,6.587094,74.882190
INCA,194.400000,1.903538,22.12560,80.0,46.678000,8.786202,55.447600
GFDL-emi,191.700000,1.877128,22.12410,80.0,44.684000,8.664759,57.440100


## CH4 budget tabel

In [60]:
df_budget_ch4 = pd.concat([df_ch4_burden.loc['CTRL'],df_ch4_lifetime.loc['CTRL'],
                           df_ch4_loss.loc['CTRL']],axis=1)
df_budget_ch4.columns = ['CH4 burden [Tg]','CH4 lifetime [yrs]','CH4 atm.loss[Tg/yr]'] 
df_budget_ch4

,CH4 burden [Tg],CH4 lifetime [yrs],CH4 atm.loss[Tg/yr]
OSLOCTM3,4994.964696,7.313568,682.972311
WACCM6-2deg,5006.379000,6.744120,742.332400
INCA,4994.500000,9.087993,549.571300
GFDL-emi,4935.000000,8.475615,582.258600


# Other results/checks:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [61]:
df_ch4_burden_per_conc  = df_ch4_burden.loc['deltaCH4']/df_ch4_surfconc.loc['deltaCH4']
df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc


OSLOCTM3       2.758462
WACCM6-2deg    2.774998
INCA           2.755102
GFDL-emi            NaN
Name: ch4_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

Add explanation... 

In [62]:
df_w_o_feedback =df_ch4_lifetime.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback


OSLOCTM3       2.651321
WACCM6-2deg    2.430315
INCA           3.298605
GFDL-emi            NaN
dtype: float64

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [63]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux/df_w_o_feedback
df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_factor_ch4


OSLOCTM3       1.307753
WACCM6-2deg    1.402692
INCA           1.212466
GFDL-emi            NaN
Name: feedback_factor_ch4, dtype: float64

## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [64]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['deltaH2']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

OSLOCTM3       0.350572
WACCM6-2deg    0.349781
INCA           0.349624
GFDL-emi       0.344523
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [65]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CTRL']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2


OSLOCTM3       5.163671
WACCM6-2deg    5.074991
INCA           5.444529
GFDL-emi       5.448486
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

In [66]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

OSLOCTM3       0.973432
WACCM6-2deg    0.981648
INCA           0.980149
GFDL-emi       0.931234
Name: feedback_factor_h2, dtype: float64